In [3]:
import pandas as pd

In [4]:
train_ds= pd.read_csv('/kaggle/input/playground-series-s4e7/train.csv')
test_ds=pd.read_csv('/kaggle/input/playground-series-s4e7/test.csv')

In [5]:
train_ds.dtypes
print(train_ds.shape)
print(test_ds.shape)
train_y=train_ds["Response"]

(11504798, 12)
(7669866, 11)


In [6]:
train_ds=train_ds.drop(["Response"],axis="columns")
train_ds=train_ds.drop(["id"],axis="columns")

In [7]:
train_ds.dtypes

Gender                   object
Age                       int64
Driving_License           int64
Region_Code             float64
Previously_Insured        int64
Vehicle_Age              object
Vehicle_Damage           object
Annual_Premium          float64
Policy_Sales_Channel    float64
Vintage                   int64
dtype: object

In [8]:
test_ds=test_ds.drop(["id"],axis="columns")

In [9]:
def unique_values(df):
  for column in test_ds.columns:
    print(column,test_ds[column].unique())

In [10]:
train_ds.isna().sum()

Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          0
Annual_Premium          0
Policy_Sales_Channel    0
Vintage                 0
dtype: int64

In [11]:
train_ds.dropna(inplace=True)

In [12]:
print(unique_values(train_ds))

Gender ['Female' 'Male']
Age [20 47 22 51 23 21 72 33 67 34 25 24 45 65 40 35 26 41 66 31 39 80 52 28
 50 59 57 53 79 49 44 42 74 62 43 56 61 76 46 38 30 71 27 54 75 70 55 32
 48 29 69 73 60 58 63 37 36 64 68 77 78 82 83 81 84 85]
Driving_License [1 0]
Region_Code [47. 28. 43. 19. 30. 20. 37. 48. 29. 11. 21. 10.  0. 36. 41. 45. 35.  8.
 40. 39. 32. 34.  2.  6. 31. 49.  9. 46. 14. 18. 12. 15.  3.  4. 50. 22.
  7. 17. 33. 25. 24. 16. 27.  5. 13. 38. 23. 44. 26.  1. 42. 51. 52.]
Previously_Insured [0 1]
Vehicle_Age ['< 1 Year' '1-2 Year' '> 2 Years']
Vehicle_Damage ['No' 'Yes']
Annual_Premium [ 2630. 37483. 24502. ... 53069. 19363. 60767.]
Policy_Sales_Channel [160. 124.  26. 152.  29.   1. 122. 156.  55.  25.  52.  21. 153.  24.
 151.   4.  60.  11. 163.  54.  31. 157. 154.  10.   7. 158.   3.  13.
  61. 129. 120. 107. 147. 130.   8.  91.  23.  15. 155.  22.  14.  30.
 135. 116. 148. 109.  42.  12. 150.  32. 125. 145.  16.  65. 127.  37.
  45. 131. 121.  94. 103.  18. 106.  88. 113. 111.

In [13]:
train_ds["Gender"].replace({'Female':1,'Male':0},inplace=True)
train_ds["Vehicle_Damage"].replace({'Yes':1,'No':0},inplace=True)
test_ds["Gender"].replace({'Female':1,'Male':0},inplace=True)
test_ds["Vehicle_Damage"].replace({'Yes':1,'No':0},inplace=True)

/tmp/ipykernel_114/4165247183.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_ds["Gender"].replace({'Female':1,'Male':0},inplace=True)
/tmp/ipykernel_114/4165247183.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_ds["Gender"].replace({'Female':1,'Male':0},inplace=True)
/tmp/ipyke

In [14]:
train_ds=pd.get_dummies(train_ds,columns=['Vehicle_Age'],dtype=int)

In [15]:
test_ds=pd.get_dummies(test_ds,columns=['Vehicle_Age'],dtype=int)

In [16]:
scaled_columns=["Age","Region_Code","Annual_Premium","Policy_Sales_Channel","Vintage"]

In [17]:
combined=pd.concat([train_ds,test_ds],ignore_index=True)

In [18]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(combined[scaled_columns])
train_ds[scaled_columns]=scaler.transform(train_ds[scaled_columns])
test_ds[scaled_columns]=scaler.transform(test_ds[scaled_columns])

In [19]:
import tensorflow as tf
from tensorflow import keras

2024-07-24 21:39:35.855555: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 21:39:35.855642: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 21:39:35.859494: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [21]:
from tensorflow.keras.layers import Dense, LeakyReLU
model1=keras.Sequential([
    keras.layers.Dense(32,input_shape=(train_ds.shape[1],),activation="relu"),
    keras.layers.Dense(16),
    LeakyReLU(alpha=0.1),
    keras.layers.Dense(32,activation="relu"),
    keras.layers.Dense(16),
    LeakyReLU(alpha=0.1),
    keras.layers.Dense(1,activation="sigmoid")
    ]
                      )

In [30]:
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam
initial_learning_rate = 0.01
model1.compile(
    optimizer=Adam(learning_rate=initial_learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Define the learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

callback = LearningRateScheduler(scheduler)

# Train the model
history = model1.fit(
    train_ds,
    train_y,
    epochs=10,
    callbacks=[callback],
    batch_size=128
)

Epoch 1/10
89882/89882 ━━━━━━━━━━━━━━━━━━━━ 202s 2ms/step - accuracy: 0.8770 - loss: 0.2665 - learning_rate: 0.0100
Epoch 2/10
89882/89882 ━━━━━━━━━━━━━━━━━━━━ 176s 2ms/step - accuracy: 0.8769 - loss: 0.2662 - learning_rate: 0.0100
Epoch 3/10
89882/89882 ━━━━━━━━━━━━━━━━━━━━ 181s 2ms/step - accuracy: 0.8770 - loss: 0.2659 - learning_rate: 0.0100
Epoch 4/10
89882/89882 ━━━━━━━━━━━━━━━━━━━━ 188s 2ms/step - accuracy: 0.8769 - loss: 0.2662 - learning_rate: 0.0100
Epoch 5/10
89882/89882 ━━━━━━━━━━━━━━━━━━━━ 178s 2ms/step - accuracy: 0.8772 - loss: 0.2658 - learning_rate: 0.0100
Epoch 6/10
89882/89882 ━━━━━━━━━━━━━━━━━━━━ 183s 2ms/step - accuracy: 0.8773 - loss: 0.2656 - learning_rate: 0.0100
Epoch 7/10
89882/89882 ━━━━━━━━━━━━━━━━━━━━ 185s 2ms/step - accuracy: 0.8770 - loss: 0.2659 - learning_rate: 0.0100
Epoch 8/10
89882/89882 ━━━━━━━━━━━━━━━━━━━━ 185s 2ms/step - accuracy: 0.8771 - loss: 0.2659 - learning_rate: 0.0100
Epoch 9/10
89882/89882 ━━━━━━━━━━━━━━━━━━━━ 187s 2ms/step - accuracy: 0.

In [31]:
predict=model1.predict(test_ds)

239684/239684 ━━━━━━━━━━━━━━━━━━━━ 316s 1ms/step


In [34]:
pred=pd.DataFrame(predict)
samp=pd.read_csv("/kaggle/input/playground-series-s4e7/sample_submission.csv")
datasets=pd.concat([samp["id"],pred],axis=1)
datasets.columns=["id","Response"]
datasets.to_csv("sample_prediction_1.csv",index=False)